# Parte I

## Parameters

In [1]:
from pathlib import Path

BASE_DIR = Path('/Users/efraflores/Downloads')
FILE_NAME = 'practica_1.csv'

## Import

In [2]:
import pandas as pd

df = pd.read_csv(BASE_DIR.joinpath(FILE_NAME), low_memory=False)
df.sample()

,id,ao_hechos,mes_hechos,fecha_hechos,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,colonia_hechos,alcaldia_hechos,fecha_inicio,mes_inicio,ao_inicio,calle_hechos,calle_hechos2,longitud,latitud,geopoint
574960,600224,2019,Mayo,14/05/2019 03:00,ABUSO DE CONFIANZA,DELITO DE BAJO IMPACTO,INVESTIGACIÃN EN COYOACÃN,COY-5,UI-2CD,EL VERGEL,IZTAPALAPA,14/05/2019 17:00,Mayo,2019,AVENIDA TLAHUAC,PERIFERICO,-99.07578582,19.318471,"19.3184712729,-99.0757858187"


## Data wrangling

### Accents

In [3]:
accents_dict = {'Ã\x81':'Á','Ã\x89':'É','Ã\x8d':'Í','Ã\x93':'Ó','Ã\x9a':'Ú','Ã\x91':'Ñ'}

for col in ['delito','categoria_delito','fiscalia','colonia_hechos','alcaldia_hechos','calle_hechos']:
    for x,y in accents_dict.items():
        df[col] = df[col].astype(str).str.replace(x,y)

df.sample()

,id,ao_hechos,mes_hechos,fecha_hechos,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,colonia_hechos,alcaldia_hechos,fecha_inicio,mes_inicio,ao_inicio,calle_hechos,calle_hechos2,longitud,latitud,geopoint
515673,538382,2019,Abril,01/04/2019 06:00,ROBO DE OBJETOS,DELITO DE BAJO IMPACTO,INVESTIGACIÓN EN GUSTAVO A. MADERO,GAM-6,UI-1CD,GUADALUPE INSURGENTES,GUSTAVO A MADERO,01/04/2019 12:02,Abril,2019,ROBERTO GAYOL,ESQUINA EXCELSIOR,-99.13095395,19.474632,"19.4746321655,-99.1309539525"


### Fecha_hechos

In [4]:
df.loc[0,'fecha_hechos']

'05/01/2016 17:00'

In [5]:
pd.to_datetime(df.loc[0,'fecha_hechos'], dayfirst=True)

Timestamp('2016-01-05 17:00:00')

In [6]:
date_errors = []
for x in df['fecha_hechos']:
    try:
        pd.to_datetime(x, dayfirst=True)
    except:
        date_errors.append(x)

date_errors[:3]

['martes| 12 de julio de 2016| 20:35:00 ()',
 'domingo| 10 de febrero de 2019| 3:00:00 ()',
 'jueves| 12 de mayo de 2016| 17:00:00 ()']

In [7]:
date_errors[15]

'jueves| 10 de diciembre de 2015| 12:00:00 ()'

In [8]:
import re

aux = re.findall('\|\s(.+)\(\)', date_errors[15])[0].replace('|','')
aux

'10 de diciembre de 2015 12:00:00 '

In [9]:
month_dict = {'ene':'jan','abr':'apr','ago':'aug','dic':'dec'}

aux_day = re.findall('\d{,2}', aux)[0]
aux_month = month_dict[re.findall('de\s(.{3})', aux)[0]]
aux_year = re.findall('\d{4}', aux)[0]
aux_time = re.findall('\d{,2}\:\d{,2}', aux)[0]

print(aux_day+'/'+aux_month+'/'+aux_year+' '+aux_time)

10/dec/2015 12:00


In [10]:
date_correct = []
for x in df['fecha_hechos']:
    try:
        date_correct.append(pd.to_datetime(str(x), dayfirst=True))
        
    except:
        full_date = re.findall('\|\s(.+)\(\)', str(x))[0]

        aux_day = re.findall('\d{,2}', full_date)[0]
        aux_year = re.findall('\d{4}', full_date)[0]
        aux_time = re.findall('\d{,2}:\d{,2}', aux)[0]

        aux_month = re.findall('de\s(.{3})', full_date)[0]

        if aux_month in month_dict.keys(): 
            aux_month = month_dict[aux_month]

        concat_date = aux_day+'/'+aux_month+'/'+aux_year+' '+aux_time

        date_correct.append(pd.to_datetime(concat_date, format='%d/%b/%Y %H:%M'))

date_correct[:3]

[Timestamp('2016-01-05 17:00:00'),
 Timestamp('2017-11-01 16:40:00'),
 Timestamp('2015-12-30 20:00:00')]

In [11]:
df['fecha_hechos'] = date_correct
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774952 entries, 0 to 774951
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id                    774952 non-null  int64         
 1   ao_hechos             774952 non-null  object        
 2   mes_hechos            774952 non-null  object        
 3   fecha_hechos          774952 non-null  datetime64[ns]
 4   delito                774952 non-null  object        
 5   categoria_delito      774952 non-null  object        
 6   fiscalia              774952 non-null  object        
 7   agencia               774952 non-null  object        
 8   unidad_investigacion  774952 non-null  object        
 9   colonia_hechos        774952 non-null  object        
 10  alcaldia_hechos       774952 non-null  object        
 11  fecha_inicio          774952 non-null  object        
 12  mes_inicio            774952 non-null  object        
 13 

# Parte II

### Greater than 2013

In [12]:
df = df[df['fecha_hechos'].dt.year > 2013].copy()

### Normalize crimes

In [13]:
df['categoria_delito'].value_counts()

DELITO DE BAJO IMPACTO                                     608483
HECHO NO DELICTIVO                                          46327
ROBO DE VEHÍCULO CON Y SIN VIOLENCIA                        40975
ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA        33090
ROBO A NEGOCIO CON VIOLENCIA                                11509
ROBO A REPARTIDOR CON Y SIN VIOLENCIA                        7177
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA        5401
HOMICIDIO DOLOSO                                             3806
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO                3032
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA      2827
VIOLACIÓN                                                    1828
ROBO A CASA HABITACIÓN CON VIOLENCIA                         1826
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA      1402
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA                 723
ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA                      408
SECUESTRO 

In [14]:
df['categoria_delito'] = df['categoria_delito'].str.split().str[0]
df['categoria_delito'].value_counts()

DELITO       608483
ROBO         105338
HECHO         46327
HOMICIDIO      3806
LESIONES       3032
VIOLACIÓN      1828
SECUESTRO        90
Name: categoria_delito, dtype: int64

### Date variables

In [15]:
df['quarter'] = df['fecha_hechos'].dt.quarter
df['weekday'] = df['fecha_hechos'].dt.day_name().str[:3]
df['weekend'] = df['weekday'].map(lambda x: x in ('Fri','Sat','Sun'))
df['day_of_event'] = df['fecha_hechos'].dt.day
df.sample()

,id,ao_hechos,mes_hechos,fecha_hechos,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,colonia_hechos,...,ao_inicio,calle_hechos,calle_hechos2,longitud,latitud,geopoint,quarter,weekday,weekend,day_of_event
535782,559344,2018,Mayo,2018-05-03 18:59:00,FRAUDE,DELITO,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-1,UI-1SD,NARVARTE,...,2018,LUZ SAVIÑON NO.9 DESPACHO 602 Y 702 PISO 6 Y 7,NaN,-99.15959595,19.39123,"19.3912300277,-99.1595959538",2,Thu,False,3


### Normalize zone

In [42]:
df['zone'] = df['alcaldia_hechos'].map({'GUSTAVO A MADERO': 'ZONA NORTE',
                                    'IZTACALCO': 'ZONA NORTE',
                                    'IZTAPALAPA': 'ZONA ORIENTE',
                                    'BENITO JUAREZ': 'ZONA SUR',
                                    'CUAUHTEMOC': 'ZONA CENTRO PONIENTE',
                                    'LA MAGDALENA CONTRERAS': 'ZONA SUR',
                                    'AZCAPOTZALCO': 'ZONA CENTRO PONIENTE',
                                    'MIGUEL HIDALGO': 'ZONA CENTRO PONIENTE',
                                    'VENUSTIANO CARRANZA': 'ZONA NORTE',
                                    'ALVARO OBREGON': 'ZONA CENTRO PONIENTE',
                                    'MILPA ALTA': 'ZONA ORIENTE',
                                    'TLALPAN': 'ZONA SUR',
                                    'COYOACAN': 'ZONA SUR',
                                    'XOCHIMILCO': 'ZONA ORIENTE',
                                    'CUAJIMALPA DE MORELOS': 'ZONA CENTRO PONIENTE',
                                    'TLAHUAC': 'ZONA ORIENTE'})
df['zone'].value_counts(1)               

ZONA CENTRO PONIENTE    0.357883
ZONA SUR                0.230643
ZONA ORIENTE            0.209454
ZONA NORTE              0.202020
Name: zone, dtype: float64

# Parte III

## TAD

In [43]:
df = df[df['fecha_hechos'].dt.year>=2016].copy()
tad = df.copy()

### Date variables

In [45]:
tad['date'] = tad['fecha_hechos'].dt.date
tad['hour'] = tad['fecha_hechos'].dt.hour.astype(str).str.zfill(2)
tad['fecha_hechos'].dt.year.value_counts().sort_index()

2016    186041
2017    215858
2018    239469
2019    109437
Name: fecha_hechos, dtype: int64

In [46]:
backup = tad.copy()
backup.sample()

,id,ao_hechos,mes_hechos,fecha_hechos,delito,categoria_delito,fiscalia,agencia,unidad_investigacion,colonia_hechos,...,longitud,latitud,geopoint,quarter,weekday,weekend,day_of_event,zone,date,hour
373619,390130,2018,Mayo,2018-05-12 01:30:00,DENUNCIA DE HECHOS,HECHO,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-3,UI-3SD,NARVARTE,...,-99.15076831,19.399852,"19.3998524595,-99.1507683088",2,Sat,True,12,ZONA SUR,2018-05-12,01


### Crime number by date-hour

In [18]:
tad = tad.pivot_table(index='date', columns='hour', values='id', aggfunc='count')
tad.head()

hour,00,01,02,03,04,05,06,07,08,09,...,14,15,16,17,18,19,20,21,22,23
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,19.0,16.0,10.0,10.0,8.0,7.0,8.0,11.0,9.0,18.0,...,13.0,13.0,16.0,24.0,9.0,17.0,18.0,20.0,6.0,12.0
2016-01-02,9.0,7.0,8.0,6.0,2.0,4.0,4.0,13.0,17.0,18.0,...,11.0,17.0,23.0,20.0,19.0,18.0,24.0,19.0,24.0,22.0
2016-01-03,4.0,16.0,7.0,7.0,5.0,7.0,6.0,9.0,12.0,15.0,...,27.0,18.0,16.0,20.0,24.0,24.0,21.0,19.0,14.0,11.0
2016-01-04,7.0,6.0,7.0,7.0,6.0,8.0,16.0,16.0,20.0,30.0,...,26.0,27.0,30.0,35.0,38.0,21.0,39.0,19.0,17.0,12.0
2016-01-05,8.0,7.0,4.0,5.0,3.0,7.0,7.0,12.0,9.0,17.0,...,33.0,34.0,31.0,25.0,26.0,25.0,34.0,26.0,16.0,14.0


### Every date

In [19]:
pd.date_range(start='2021-05-05', end='2021-05-10')

DatetimeIndex(['2021-05-05', '2021-05-06', '2021-05-07', '2021-05-08',
               '2021-05-09', '2021-05-10'],
              dtype='datetime64[ns]', freq='D')

In [20]:
pd.DataFrame(pd.date_range(start='2021-05-05', end='2021-05-10'),columns=['date']).set_index('date')

""
date
2021-05-05
2021-05-06
2021-05-07
2021-05-08
2021-05-09
2021-05-10


In [21]:
tad = pd.DataFrame(pd.date_range(start=tad.index.min(), end=tad.index.max()),columns=['date']).set_index('date').join(tad).fillna(0)
tad.head()

,00,01,02,03,04,05,06,07,08,09,...,14,15,16,17,18,19,20,21,22,23
date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,19.0,16.0,10.0,10.0,8.0,7.0,8.0,11.0,9.0,18.0,...,13.0,13.0,16.0,24.0,9.0,17.0,18.0,20.0,6.0,12.0
2016-01-02,9.0,7.0,8.0,6.0,2.0,4.0,4.0,13.0,17.0,18.0,...,11.0,17.0,23.0,20.0,19.0,18.0,24.0,19.0,24.0,22.0
2016-01-03,4.0,16.0,7.0,7.0,5.0,7.0,6.0,9.0,12.0,15.0,...,27.0,18.0,16.0,20.0,24.0,24.0,21.0,19.0,14.0,11.0
2016-01-04,7.0,6.0,7.0,7.0,6.0,8.0,16.0,16.0,20.0,30.0,...,26.0,27.0,30.0,35.0,38.0,21.0,39.0,19.0,17.0,12.0
2016-01-05,8.0,7.0,4.0,5.0,3.0,7.0,7.0,12.0,9.0,17.0,...,33.0,34.0,31.0,25.0,26.0,25.0,34.0,26.0,16.0,14.0


### Melt

In [22]:
tad = tad.melt(ignore_index=False, var_name='hour', value_name='crimes').fillna(0)
tad = tad.reset_index().sort_values(by=['date','hour']).set_index(['date','hour'])
tad.head()

crimes
date       hour        
2016-01-01 00      19.0
           01      16.0
           02      10.0
           03      10.0
           04       8.0

### Shift

In [31]:
for i in range(1,529):
    tad[f'{i}_hrs_ago'] = tad['crimes'].shift(i)

tad.dropna(inplace=True)

tad.head(11).style.background_gradient(cmap='Blues').format('{:.1f}')

In [30]:
tad.shape

(30120, 529)

## By zone

In [51]:
def multishift(data, zone_col, shifts, **pivot_args):
    df = pd.DataFrame()
    for zone in set(data[zone_col]):
        df_id = data.set_index(zone_col).loc[zone,:].pivot_table(**pivot_args)
        tot_dates = pd.DataFrame(pd.date_range(start=df_id.index.min(), end=df_id.index.max()),columns=['date']).set_index('date').join(df_id).fillna(0)
        tad = tot_dates.melt(ignore_index=False, var_name='hour', value_name='crimes').fillna(0)
        tad = tad.reset_index().sort_values(by=['date','hour']).set_index(['date','hour'])
        for i in shifts:
            tad[f'{i}_hrs_ago'] = tad['crimes'].shift(i)

        tad.dropna(inplace=True)
        tad['zone'] = zone
        df = df.append(tad, ignore_index=False)
    return df


by_zone = multishift(backup, 'zone', range(1,529), index='date', columns='hour', values='id', aggfunc='count')
by_zone = by_zone.reset_index().set_index(['zone','date','hour'])
by_zone.tail()

crimes  1_hrs_ago  2_hrs_ago  3_hrs_ago  4_hrs_ago  \
zone     date       hour                                                       
ZONA SUR 2019-06-30 19       3.0        2.0        1.0        2.0        2.0   
                    20       4.0        3.0        2.0        1.0        2.0   
                    21       0.0        4.0        3.0        2.0        1.0   
                    22       0.0        0.0        4.0        3.0        2.0   
                    23       0.0        0.0        0.0        4.0        3.0   

                          5_hrs_ago  6_hrs_ago  7_hrs_ago  8_hrs_ago  \
zone     date       hour                                               
ZONA SUR 2019-06-30 19          4.0        4.0        2.0        1.0   
                    20          2.0        4.0        4.0        2.0   
                    21          2.0        2.0        4.0        4.0   
                    22          1.0        2.0        2.0        4.0   
                    23          2.0        1.0        2.0        2.0   

                          9_hrs_ago  ...  519_hrs_ago  520_hrs_ago  \
zone     date       hour             ...                             
ZONA SUR 2019-06-30 19          3.0  ...          2.0          5.0   
                    20          1.0  ...          4.0          2.0   
                    21          2.0  ...          3.0          4.0   
                    22          4.0  ...          2.0          3.0   
                    23          4.0  ...          6.0          2.0   

                          521_hrs_ago  522_hrs_ago  523_hrs_ago  524_hrs_ago  \
zone     date       hour                                                       
ZONA SUR 2019-06-30 19            4.0          2.0          1.0          6.0   
                    20            5.0          4.0          2.0          1.0   
                    21            2.0          5.0          4.0          2.0   
                    22            4.0          2.0          5.0          4.0   
                    23            3.0          4.0          2.0          5.0   

                          525_hrs_ago  526_hrs_ago  527_hrs_ago  528_hrs_ago  
zone     date       hour                                                      
ZONA SUR 2019-06-30 19            8.0          6.0          8.0          7.0  
                    20            6.0          8.0          6.0          8.0  
                    21            1.0          6.0          8.0          6.0  
                    22            2.0          1.0          6.0          8.0  
                    23            4.0          2.0          1.0          6.0  

[5 rows x 529 columns]

## Model

### Preprocessing

In [52]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = by_zone.iloc[:,1:].copy()
y = by_zone['crimes'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    train_size=0.77,
                                                    random_state=22)
mm_x = MinMaxScaler()

### Linear Regression

In [54]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()

model_reg = Pipeline(steps=[('scaler', mm_x),
                            ('model', linear_reg)])

print(f"Score: {'{:.2%}'.format(model_reg.fit(X_train,y_train).score(X_test,y_test))}")
print(f"Training score: {'{:.2%}'.format(model_reg.score(X_train,y_train))}")
#The most relevant features to the model_reg
coef = pd.DataFrame(zip(X.columns,model_reg[1].coef_)).sort_values(1,0,0).reset_index(drop=True)
coef.head().append(coef.tail())

Score: 65.594%
Training score: 64.730%


,0,1
0,336_hrs_ago,21.410530
1,168_hrs_ago,18.542321
2,504_hrs_ago,17.796361
3,2_hrs_ago,13.217394
4,1_hrs_ago,12.843264
523,356_hrs_ago,-1.994382
524,386_hrs_ago,-2.120680
525,458_hrs_ago,-2.139261
526,147_hrs_ago,-2.901772
527,409_hrs_ago,-2.916373


### Forest Regressor

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_jobs=-1,random_state=22)

model_forest = Pipeline(steps=[('scaler', mm_x),
                               ('model', forest)])

print(f"Score: {'{:.2%}'.format(model_forest.fit(X_train,y_train).score(X_test,y_test))}")
print(f"Training score: {'{:.2%}'.format(model_forest.score(X_train,y_train))}")

#The most relevant features to the model_forest
pd.DataFrame(zip(X.columns,model_forest[1].feature_importances_)).sort_values(1,0,0).head(10).reset_index(drop=True)

Score: 66.443%
Training score: 95.094%


,0,1
0,336_hrs_ago,0.241353
1,168_hrs_ago,0.195664
2,504_hrs_ago,0.101372
3,24_hrs_ago,0.022600
4,1_hrs_ago,0.016710
5,2_hrs_ago,0.010040
6,144_hrs_ago,0.006651
7,480_hrs_ago,0.006032
8,192_hrs_ago,0.005943
9,312_hrs_ago,0.004618
